# Exercise A1: IFC Dashboard
41934 Advanced Building Information Modeling (BIM), E23

Kaare G. S. Hansen, s214282 - DTU



In [10]:
import os
from collections import namedtuple
import numpy as np
import pandas as pd
import ifcopenshell
import ifcopenshell.util.placement
import ifcopenshell.geom

import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Load IFC-file

In [22]:
model_dir = "/Users/Kaare/My Drive/DTU/Kurser/Videregaaende BIM - 41934/IFC-models"

# model_file = "SkyLab/LLYN - ARK - Space.ifc"
model_file = "SkyLab/LLYN - STRU.ifc"
model_path = os.path.join(model_dir, model_file)
# model_path = model_dir + '/' + model_file

print(f"File path: {model_path}")

model = ifcopenshell.open(model_path)
ModelHasColor = True
print(f"Model schema: {model.schema}")

File path: /Users/Kaare/My Drive/DTU/Kurser/Videregaaende BIM - 41934/IFC-models\SkyLab/LLYN - STRU.ifc
Model schema: IFC4


## Select data

In [79]:
slabs = model.by_type('IfcSlab')

In [117]:
slab_DE = []
for slab in slabs:
    info = slab.get_info()
    objectType = info['ObjectType']
    if not 'Floor:DE' in objectType: continue

    slab_DE.append(slab)
    # print(info)

    psets = ifcopenshell.util.element.get_psets(slab)
    # print(psets.keys())
    Pset_SlabCommon = psets['Pset_SlabCommon']
    print("Reference", Pset_SlabCommon['Reference'])
    # print(psets)
    # print(pset_ReinforcementBarPitchOfSlab)


Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference DE22
Reference 

## Rendering

### Functions

In [23]:
def getElementShapes(entities):
    settings = ifcopenshell.geom.settings()

    element_shapes = []
    for entity in entities:
        try:
            element = entity
            shape = ifcopenshell.geom.create_shape(settings, element)
            element_shapes.append(shape)
        except Exception as e:
            print(element.is_a())
            print(e)

    print(f"Count: ({len(element_shapes)}/{len(entities)})")

    return element_shapes

In [74]:
RenderData = namedtuple('RenderData', ['mesh_list', 'edges_list'] )

def makeRenderData(element_shapes):
    mesh_list = []
    edges_list = []

    for shape in element_shapes:
        geometry = shape.geometry

        verts = np.array(geometry.verts)
        verts = np.reshape(verts, (-1, 3))

        faces = np.array(geometry.faces)
        faces = np.reshape(faces, (-1, 3))

        edges = np.array(geometry.edges)
        edges = np.reshape(edges, (-1, 2))

        #_normals = np.array(geometry.normals)
        #_normals = np.reshape(_normals, (-1, 3))

        #if len(geometry.materials) > 1:
        #    print("Ids:", shape.geometry.material_ids, "styles:", len(geometry.materials))
        
        #print(f"{len(shape.geometry.material_ids)}, {len(shape.geometry.materials)}, {len(verts)}, {len(faces)}")

        vertices = o3d.utility.Vector3dVector(verts)
        triangles = o3d.utility.Vector3iVector(faces)
        #normals = o3d.utility.Vector3dVector(_normals)
        #print("Normals", len(geometry.normals)) # Det ser ikke ud til at der kommer normals fra Ifc-filen

        #transformation_matrix = o3d.utility.Matrix3dVector(shape.transformation.matrix.data)

        mat = np.array(shape.transformation.matrix.data, copy=True)
        
        mat = np.reshape(mat, (-1, 3))
        transform = mat[:3,:].T
        translate = mat[3,:]
        #print(np.shape(mat))
        
        T = np.eye(4)
        T[:3, :3] = transform
        #T[0, 3] = 1

        mesh_np = o3d.geometry.TriangleMesh(vertices, triangles)
        mesh_np.transform(T)
        mesh_np.translate(translate)

        #mesh_np.vertex_normals = normals
        #mesh_np.triangle_normals = normals
        mesh_np.compute_vertex_normals()

        # Colors
        if ModelHasColor:
            colors = np.empty((len(vertices), 3))
            # print(geometry.material_ids)
            for i, material_id in enumerate(geometry.material_ids):
                if material_id == -1: # No material!
                    color = np.array([1.0, 0.0, 1.0])
                else:
                    material = geometry.materials[material_id]
                    color = np.array(material.diffuse)
                
                face = faces[i]

                for vertexIndex in face:
                    colors[vertexIndex] = color

            mesh_np.vertex_colors = o3d.utility.Vector3dVector(colors)
            assert np.shape(colors) == np.shape(vertices)

        edges = o3d.utility.Vector2iVector(edges)
        edges_np = o3d.geometry.LineSet(vertices, edges)
        edges_np.transform(T)
        edges_np.translate(translate)

        mesh_list.append(mesh_np)
        edges_list.append(edges_np)

    print(f"Mesh list length: {len(mesh_list)}")
    return RenderData(mesh_list, edges_list)

In [75]:
def runVisualization(data: RenderData):
    #print(np.asarray(mesh_np.triangle_normals))
    print("Displaying mesh made using numpy ...")
    mesh_origin = o3d.geometry.TriangleMesh.create_coordinate_frame()
    data.mesh_list.append(mesh_origin)
    o3d.visualization.draw_geometries(data.mesh_list + data.edges_list)
    #o3d.visualization.draw_geometries(edges_list)

### Render entities

In [118]:
beams = model.by_type('IfcBeam')
slabs = model.by_type('IfcSlab')

# entities = beams + slabs
entities = slab_DE
len(entities)

321

In [119]:
elementShapes = getElementShapes(entities)
print(f"Created element shapes")
renderData = makeRenderData(elementShapes)
print(f"Created render data")

Count: (321/321)
Created element shapes
Mesh list length: 321
Created render data


In [120]:
runVisualization(renderData)
print(f"Visualization exited")

Displaying mesh made using numpy ...
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The req